In [12]:
from datasets import load_dataset
from utils import jpeg_compress, webp_compress, pad, crop, nn_compress, hific_lo_compress
import compressai
import evaluate
from transformers import AutoFeatureExtractor, SegformerForSemanticSegmentation
from piq import LPIPS
import torch
from torch.nn import MSELoss
import numpy as np

In [2]:
def jpeg_compress_ade(sample):
    img,bpp = jpeg_compress(sample['image'])
    sample['image'] = img
    sample['bpp' ] = bpp
    return sample
    
def webp_compress_ade(sample):
    img,bpp = webp_compress(sample['image'])
    sample['image'] = img
    sample['bpp' ] = bpp
    return sample

net_mbt2018 = compressai.zoo.mbt2018(quality=1, pretrained=True).eval().to("cpu")
def mbt2018_compress_ade(sample):
    img,bpp = nn_compress(sample['image'],net_mbt2018,"cpu")
    sample['image'] = img
    sample['bpp'] = bpp
    return sample

def hific_lo_compress_ade(sample):
    img,bpp = hific_lo_compress(sample['image'])
    sample['image'] = img
    sample['bpp' ] = bpp
    return sample

In [3]:
image_compression_methods = [
    jpeg_compress_ade,
    webp_compress_ade,
    mbt2018_compress_ade,
    hific_lo_compress_ade
]

In [4]:
ade20k = [load_dataset("scene_parse_150",split='validation')]

In [5]:
for method in image_compression_methods:
    ade20k.append(ade20k[0].map(method))

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [8]:
image_bpp = [np.mean(method['bpp']) for method in ade20k[1:]]
image_bpp

[0.2616756840709221,
 0.13477392121864096,
 0.13477048131961145,
 0.025415553682810475]

In [9]:
lpips_metric = LPIPS()
mse_metric = MSELoss()

/home/dan/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dan/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
%%time
lpips = []
mse = []
for method in ade20k:
    method = method.with_format("torch")
    lpips.append([])
    mse.append([])
    for i_sample,sample in enumerate(ade20k[0].with_format("torch")):
        compressed_sample = method[i_sample]
        reference = sample['image'].to(torch.float32).permute(2,0,1).unsqueeze(0)
        distorted = compressed_sample['image'].to(torch.float32).permute(2,0,1).unsqueeze(0)
        lpips[-1].append(lpips_metric(reference,distorted).detach().item())
        mse[-1].append(mse_metric(reference,distorted).detach().item())

CPU times: user 19h 52min 1s, sys: 57min 42s, total: 20h 49min 43s
Wall time: 53min 22s


In [14]:
PSNR = [[20*np.log10(255)-10*np.log10(d) for d in m] for m in mse]
PSNR = [np.mean(d) for d in PSNR]
PSNR

/tmp/ipykernel_1318356/986530016.py:1: RuntimeWarning: divide by zero encountered in log10
  PSNR = [[20*np.log10(255)-10*np.log10(d) for d in m] for m in mse]


[inf,
 23.853026115844035,
 25.599155706072825,
 28.05273724344183,
 27.70333011289383]

In [15]:
neg_log_lpips = [[-10*np.log10(d) for d in m] for m in lpips]
neg_log_lpips = [np.mean(d) for d in neg_log_lpips]
neg_log_lpips

/tmp/ipykernel_1318356/1126017225.py:1: RuntimeWarning: divide by zero encountered in log10
  neg_log_lpips = [[-10*np.log10(d) for d in m] for m in lpips]


[inf,
 7.134667172112102,
 7.959768308675015,
 8.992722011607638,
 11.813737897576733]